In [49]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from pandas.api.types import CategoricalDtype
from sklearn.neighbors import NearestNeighbors

In [3]:
ratings = pd.read_csv('ratings.csv')
ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [5]:
books = pd.read_csv('books.csv')
books

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...


In [9]:
user_index = ratings.user_id.unique()
books_index = ratings.book_id.unique()

In [15]:
rows = ratings['user_id'].astype(CategoricalDtype(categories=user_index)).cat.codes
cols = ratings['book_id'].astype(CategoricalDtype(categories=books_index)).cat.codes

In [17]:
matrix = sparse.csr_matrix((ratings.rating, (rows, cols)), shape=(len(user_index), len(books_index)))

In [19]:
matrix = matrix.toarray()
matrix

array([[5, 0, 0, ..., 0, 0, 0],
       [0, 4, 5, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [27]:
matrix.shape

(53424, 10000)

In [39]:
for ind, score in enumerate(matrix[50]):
    if score>3:
        books_id = books_index[ind]
        print(score, books[books['book_id']==books_id].original_title.values[0])

4 La sombra del viento
5 The Sun Also Rises
4 The Catcher in the Rye
4 Slaughterhouse-Five, or The Children's Crusade: A Duty-Dance with Death 
4 Life of Pi
4 Animal Farm: A Fairy Story
4 Nineteen Eighty-Four
4 Never Let Me Go
4 The Sea
4 Of Mice and Men 
4 Das Parfum. Die Geschichte eines Mörders
4 The Secret History
4 Motherless Brooklyn
4 In Cold Blood
4 Kitchen Confidential: Adventures in the Culinary Underbelly
4 Franny and Zooey
4 The Poisonwood Bible
4 Frankenstein; or, The Modern Prometheus
4 Howl and Other Poems 
4 The Hours
4 Under the Banner of Heaven: A Story of Violent Faith
5 The Omnivore's Dilemma
4 Fahrenheit 451
4 The Crucible
4 The Grapes of Wrath
4 Disgrace
4 The Little Friend
4 On the Road
4 Nesnesitelná lehkost bytí
4 The Bean Trees
4 L'Amant
5 Stiff: The Curious Lives of Human Cadavers
4 Candide
4 Angela's Ashes: A Memoir
4 The Keep
4 Lolita
4 Still Life with Woodpecker
4 Bastard Out of Carolina
5 Le Petit Prince
4 Se una notte d'inverno un viaggiatore
4 Siddharth

In [47]:
viwed_arr = np.where(matrix[50]>0)[0]
viwed_arr

array([   0,   13,   34,   36,   38,   45,   52,   69,   73,   74,   85,
        103,  117,  120,  122,  125,  151,  165,  172,  179,  180,  187,
        193,  195,  205,  207,  216,  221,  222,  228,  231,  233,  267,
        274,  290,  307,  315,  325,  344,  385,  418,  422,  482,  486,
        506,  523,  532,  547,  644,  647,  693,  694,  710,  721,  722,
        723,  724,  725,  726,  727,  728,  729,  730,  731,  732,  733,
        734,  735,  736,  818,  877,  927,  970, 1006, 1098, 1280, 1356,
       1378, 1405, 1609, 2233, 2284, 2353, 2361, 2941, 2942, 2977, 3037,
       3180, 3181, 3374, 3422, 3438, 3775, 5131, 5325, 5619, 5644, 5710,
       6095, 6970, 7039, 7086], dtype=int64)

In [71]:


nbrs = NearestNeighbors(n_neighbors=10).fit(matrix)
search_vec = matrix[50]
closest_vec = matrix[nbrs.kneighbors(np.atleast_2d(search_vec))[1]]

In [73]:
recommendations = []
for vec in closest_vec[0]:
    for ind, value in enumerate(vec):
        if value>=3:
            if ind not in viwed_arr:
                recommendations.append(ind)

In [75]:
for ind in recommendations:
    book_id = books_index[ind]
    print(books[books['book_id']==book_id].original_title.values[0])

Shatter Me (Shatter Me, #1)
The Jordan Rules
The Hard Thing about Hard Things: Building A Business When There Are No Easy Answers
Freakonomics: A Rogue Economist Explores the Hidden Side of Everything
Rework
Quiet: The Power of Introverts in a World that Can't Stop Talking
From a Buick 8
The Long Walk


In [81]:
user_indexes_for_masking = np.random.randint(matrix.shape[0], size=100)

In [85]:
masked_result = {}
for usr_index in user_indexes_for_masking:
    for ind, value in enumerate(matrix[usr_index]):
        if value>=4:
            masked_result[(usr_index, ind)] = value
            matrix[usr_index][ind] = 0
            break

In [87]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=10).fit(matrix)

In [89]:
repair_grade = 0

for position in masked_result.keys():
    print(position)
    search_vec = matrix[position[0]]
    closest_vec = matrix[nbrs.kneighbors(np.atleast_2d(search_vec))[1]]
    for vec in closest_vec[0]:
        for ind, value in enumerate(vec):
            if value>=4:
                if ind not in viwed_arr:
                    recommendations.append(ind)
    for recs in recommendations:
        if recs == position[1]:
            repair_grade+=1
            break

(34686, 5)
(52459, 5)
(33213, 1778)
(4615, 0)
(53316, 5)
(28033, 0)
(31308, 2595)
(50650, 2618)
(5748, 31)
(52114, 15)
(42934, 12)
(32421, 12)
(43498, 36)
(34452, 188)
(8101, 78)
(34534, 2)
(1332, 5)
(312, 32)
(21521, 5)
(35028, 13)
(104, 5)
(4191, 5)
(36, 5)
(48371, 18)
(10677, 114)
(4066, 5)
(6671, 5)
(24593, 18)
(19964, 142)
(886, 40)
(46649, 0)
(49352, 18)
(43113, 78)
(19844, 0)
(9238, 24)
(22182, 64)
(10741, 5)
(39372, 7)
(45664, 5)
(23755, 66)
(49202, 8)
(156, 7)
(53405, 18)
(36278, 1693)
(18492, 12)
(37720, 5)
(25201, 15)
(2788, 5)
(16879, 5)
(39365, 18)
(33388, 12)
(43236, 12)
(1932, 264)
(8593, 0)
(52460, 3027)
(20671, 18)
(41476, 7)
(24338, 7)
(3000, 104)
(34107, 18)
(35841, 5)
(21150, 12)
(47588, 7)
(26221, 65)
(16753, 12)
(1500, 18)
(48983, 27)
(11908, 15)
(6107, 32)
(30663, 18)
(6645, 109)
(25064, 28)
(5236, 73)
(39803, 7)
(1072, 59)
(36751, 12)
(37625, 18)
(43461, 54)
(50640, 27)
(25070, 5)
(14260, 32)
(578, 13)
(21404, 0)
(8036, 7)
(47285, 68)
(41604, 15)
(50671, 5)
(249

In [91]:
repair_grade/len(masked_result)

0.79